In [1]:
import pandas as pd
from pathlib import Path

In [12]:
sub_folder = "9"

results_path = Path("results_community") / sub_folder # Directory where we will store all the results
results_path.mkdir(exist_ok=True, parents=True)

In [13]:
# Write mp3 lengths typescript object (requires mutagen)
from mutagen.mp3 import MP3

audio_base_path = Path("..") / "content" / "sounds" / "community" # Path where the mp3s are
sound_lengths_path = results_path / "Sounds.ts"

audio_paths = list(audio_base_path.glob("*.mp3"))
audio_names = [path.stem for path in audio_paths]
print("Found", len(audio_paths), "files")

with open(sound_lengths_path, "w", encoding="utf-8") as sound_file:
    for path in audio_paths:
        duration = MP3(path).info.length
        sound_file.write('    "%s": %f,\n' % (path.stem, duration))

Found 160 files


In [14]:
# Write localization key typescript object
ts_file_path = results_path / "localization.ts"

audio_grouped_names = {}
for name in audio_names:
    base_name, nr = name.rsplit("_", 1)
    if base_name not in audio_grouped_names:
        audio_grouped_names[base_name] = []
    audio_grouped_names[base_name].append(name)
    if name.endswith("_") or any(c2.islower() and c1 == "_" for c1, c2 in zip(name[:-1], name[1:])) or name[0].islower():
        print("Wrong name", name)

with open(ts_file_path, "w", encoding="utf-8") as ts_file:
    ts_file.write("const communitySounds: Record<LocalizationKey, string[]> = {\n")
    for base_name, names in audio_grouped_names.items():
        ts_file.write(f'[LocalizationKey.{base_name}]: [\n')
        ts_file.write("\n".join(f'"{name}",' for name in names))
        ts_file.write("\n],\n")
    ts_file.write("}\n")

In [15]:
# Write sound events file
sound_events_path = results_path / "community.vsndevts"

sound_template = """    "%s" =
    {
        type = "dota_src1_2d"
        volume = "1.0"
        vsnd_files = [ "sounds/community/%s.vsnd" ]
    }
""".replace("\r\n", "\n")

with open(sound_events_path, "w", encoding="utf-8") as sound_file:
    sound_file.write("<!-- kv3 encoding:text:version{e21c7f3c-8a33-41c5-9977-a76d3a32aa0d} format:generic:version{7412167c-06e9-4698-aff2-e63eb59037e7} -->\n")
    sound_file.write("{\n")
    for row in audio_names:
        sound_file.write(sound_template % (row, row))
    sound_file.write("}\n")

In [7]:
"""
# Convert to 128
from mutagen.mp3 import MP3
import os

audio_base_path = Path("..") / "content" / "sounds" / "community" # Path where the mp3s are

processed_dir = results_path / "processed"
processed_dir.mkdir(exist_ok=True, parents=True)

audio_paths = list(audio_base_path.glob("*.mp3"))
print("Found", len(audio_paths), "files")

for path in audio_paths:
    print(str(path.absolute()), "=>", str(processed_dir.absolute() / path.name))
    os.system(" ".join(["ffmpeg", "-i", f'"{str(path.absolute())}"', "-vn", "-ar 44100", "-ac 2", "-b:a 128k", f'"{str(processed_dir.absolute() / path.name)}"']))
"""

Found 23 files
f:\TypeScript\dota-tutorial\notebooks\new_comm\General_Boo_134.mp3 => f:\TypeScript\dota-tutorial\notebooks\results_community\9\processed\General_Boo_134.mp3
f:\TypeScript\dota-tutorial\notebooks\new_comm\General_Boo_20.mp3 => f:\TypeScript\dota-tutorial\notebooks\results_community\9\processed\General_Boo_20.mp3
f:\TypeScript\dota-tutorial\notebooks\new_comm\General_Boo_526.mp3 => f:\TypeScript\dota-tutorial\notebooks\results_community\9\processed\General_Boo_526.mp3
f:\TypeScript\dota-tutorial\notebooks\new_comm\General_Boo_76.mp3 => f:\TypeScript\dota-tutorial\notebooks\results_community\9\processed\General_Boo_76.mp3
f:\TypeScript\dota-tutorial\notebooks\new_comm\General_Cheer_134.mp3 => f:\TypeScript\dota-tutorial\notebooks\results_community\9\processed\General_Cheer_134.mp3
f:\TypeScript\dota-tutorial\notebooks\new_comm\General_Cheer_20.mp3 => f:\TypeScript\dota-tutorial\notebooks\results_community\9\processed\General_Cheer_20.mp3
f:\TypeScript\dota-tutorial\noteboo